In [1]:
# Things to do...

# see how a win looks
# get script to run when Player.log is updated...or just run after each session
# fix variable naming convention to use _s
# new repo for it

In [2]:
import os, datetime
import pandas as pd

# change these
HOURS_DIFFERENTIAL = 6 # have to subtract 6

# jump is good, not sure on others
list_of_finals = ['round_fall_mountain', 'round_hex', 'round_royalfumble', 'round_jump_showdown', 
                  'round_fall_mountain_hub_complete']

with open('totalshows.txt') as f:
    total_shows = f.read()
total_shows = int(total_shows.strip())

with open(os.path.join('data', 'session.txt')) as f:
    session_num = f.read()
session_num = int(session_num)

# if data folder doesn't exist, make it
if not os.path.exists('data'):
    os.makedirs('data')

# if data folder doesn't exist, make it
if not os.path.exists(os.path.join('data', 'archive')):
    os.makedirs(os.path.join('data', 'archive'))

In [3]:
# Notes

# find player id for round - don't think we can do anything about that
# use that as well instead of username
# only affects userEndRoundLines

# check elim time for jump_showdown - seems as though personal end times are only for...
# ...finishing races early

# figure out if there's a max number of shows that will be stored in the .log : seems the whole session

In [4]:
def cleanLines(lines):
    return [line.replace('[', '').replace(']', '').replace('>', '').strip() for line in lines]

# splits show into show data and rounds
def roundSplit(lines):
    lines = cleanLines(lines)
    splits = []
    # start of each round in highlights section
    for i, line in enumerate(lines):
        if 'Round' in line:
            splits.append(i)
    
    splits.append(0)
    # split it into rounds
    rounds = [lines[splits[i-1]:splits[i]] if i != len(splits)-1 else lines[splits[i-1]:]
              for i in range(1, len(splits))]
    
    return lines[1:splits[0]], rounds

# takes registered and connected lines
# returns start times for rounds
# (time at which game is found)
def getStartTimes(reg, conne):
    startTimes = []
    
    for i in range(len(conne)):
        registeredTime = datetime.datetime.strptime(reg[i].split('at: ')[-1], '%m/%d/%Y %H:%M:%S %p')
        connectedTime = datetime.datetime.strptime(conne[i].split(': [')[0], '%H:%M:%S.%f')
        
        d = (connectedTime - registeredTime)
        startTimes.append(registeredTime + (d - datetime.timedelta(days=d.days, hours=HOURS_DIFFERENTIAL)))
    
    return startTimes

def getTimeTaken(start, end):
    d = datetime.datetime.strptime(end, '%H:%M:%S.%f') - start
    # start time was already adjusted for HOURS_DIFFERENTIAL, so need to again
    return str(d - datetime.timedelta(days=d.days, hours=HOURS_DIFFERENTIAL))[2:] # so hours isn't included

# edit this later change
# grab the last Seasonthing in txt file - countdown in .log
# if its larger than the last one, then the season reset
def getSeason():
    return 2

# gets lines for CompletedEpisode section of a show
def getShowLines(lines, marker):
    finalLines = []
    tempLines = lines[marker:]
    for line in tempLines:
        if line == '':
            continue
        if '>' not in line and '[Round' not in line and '[Complet' not in line:
            break
        if line != "":
            finalLines.append(line)
    return finalLines

# get map info lines (qual percent and # participants)
def getExtraRoundInfoLines(possLines): # rename?
    rnds = []
    currRnd = possLines[0][1].split()[0]
    currSID = possLines[0][0]

    # get last line before map switches (and server ID)
    for i, (serverID, line) in enumerate(possLines):
        rnd = line.split()[0]
        if rnd != currRnd: 
            currRnd = rnd
            currSID = serverID
            rnds.append(possLines[i-1][1])
        elif serverID != currSID:
            currRnd = rnd
            currSID = serverID
            rnds.append(possLines[i-1][1])


    rnds.append(possLines[-1][1])      
    return rnds


# preprocessGrade1 has been retired
# remove lines in-between completed games
def preprocessGrade2(lines):
    print(len(lines))
    lines2 = []
    allGood = False
    badgeID = False
    for line in lines:
        if '[CATAPULT] Login Succeeded' in line:
            allGood = True
        if 'BadgeId:' in line:
            badgeID = True
        if badgeID:
            if '[ClientGlobalGameState] ShutdownNetworkManager' in line or '[ClientGlobalGameState] sending graceful disconnect message' in line:
                badgeID = False
                allGood = False

        if allGood:
            lines2.append(line)
    lines = lines2
    print(len(lines))
    return lines

# changes to the data extraction removed the need for this
def preprocessGrade3(lines):
    #print(len(lines))
    start_conn = -1
    to_remove = []
    in_conn = False
    for i, line in enumerate(lines):
        if "[StateConnectToGame] We're connected to the server!" in line:
            start_conn = i
            in_conn = True
        if 'reports that it is not yet ready to accept connections.' in line:
            if in_conn:
                to_remove.append([start_conn, i])
                in_conn = False
    
    temp_lines = []
    for i, line in enumerate(lines):
        for check in to_remove:
            if i >= check[0] and i <= check[1]:
                continue
            temp_lines.append(line)
            
    #print(len(temp_lines))
    return temp_lines
    

# helper function (to account for if round wraps around midnight)
def subtractHours(a, b):
    c = (datetime.datetime.strptime(b, '%H:%M:%S.%f') - datetime.datetime.strptime(a, '%H:%M:%S.%f'))
    return str(c - datetime.timedelta(days=c.days))

# save data in csvs
def saveData(show, roundsList):
    new_shows_df = pd.DataFrame(pd.Series(show)).T
    new_rounds_df = pd.DataFrame(roundsList)
    try:
        #load them
        shows_df = pd.read_csv(os.path.join('data', 'shows.csv'))
        rounds_df = pd.read_csv(os.path.join('data', 'rounds.csv'))
        
        if str(show['Start Time'])[:21] in [tm[:21] for tm in shows_df['Start Time'].tolist()]:
            return False
        
        # append
        shows_df = shows_df.append(new_shows_df, ignore_index=True)
        rounds_df = rounds_df.append(new_rounds_df, ignore_index=True)
    except: # first time
        shows_df = new_shows_df
        rounds_df = new_rounds_df
        print('first time...creating csvs')
    
    # write them to their respective files
    shows_df.to_csv(os.path.join('data', 'shows.csv'), index=False)
    rounds_df.to_csv(os.path.join('data', 'rounds.csv'), index=False)
    return True

In [5]:
with open(os.path.join('C:\\Users','Joseph','AppData','LocalLow','Mediatonic','FallGuys_client','Player.log')) as f:
    lines = f.read()
#with open(os.path.join('C:\\Users','Joseph','Downloads','Player-prev.log')) as f:
 #   lines = f.read()

lines = lines.split('\n')
lines = preprocessGrade2(lines)

35594
34574


In [6]:
# get first line of each new show and usernames used for show
prevUser = '!!!!!!!!!!!!!!!'
lookUser = True
inRound = False
inARound = False
gameMode = 'main_show'

episodeMarkers = []
usernames = []
partySizes = []

# times
reg = []
conne = []
startRoundLines = []
userEndRoundLines = []
actualEndRoundLines = []

possLines = []
gameModes = []
# to find the actual number of players that qualified (for racing rounds)
prevNumLine = ""
numLines = []

# **********************************************************
# go through lines, looking for certain things**************
# **********************************************************
for i, line in enumerate(lines):
    # for username
    if lookUser and 'Sending login request' in line:
        usernames.append(line.split(' player ')[-1].split(' networkID')[0].replace(',', ''))
        prevUser = usernames[-1]
        lookUser = False
    # for type of show (main or alternate) (also called playlist)
    elif 'Chosen Show:' in line: # appears before entering matchmaking solo/group
        gameMode = line.split(':')[-1]
    # signifies start of looking for new episode
    # get size of party
    elif 'Party Size' in line or 'Begin matchmaking solo' in line:
        if 'Begin matchmaking solo' in line:
            partySize = 1
        else:
            partySize = int(line.split(' ')[-1].strip())
    # for show start time
    elif '[QosManager] Registered' in line or 'QosManager: Registered' in line: # for registered time (date)
        to_add_reg = line
    elif "[StateConnectToGame] We're connected to the server!" in line: # for connection time
        to_add_conne = line
    # for server ID and map lines
    elif 'Received NetworkGameOptions from ' in line: 
        tmp = line.split('roundID=')[-1]
        serverID = line.split(' ')[4]
        if 'Default' not in tmp:
            possLines.append([serverID, tmp])
    # for start round times and players that qualified from previous round
    elif 'state from Countdown to Playing' in line:
        startRoundLines.append(line.split(': [')[0])
        inRound = True
        inARound = True
        # append last # players achieving obj when hit new round
        if prevNumLine != "":
            numLines.append(prevNumLine.split('=')[-1])
            prevNumLine = ""
    elif '[ClientGameSession] NumPlayersAchievingObjective=' in line: # for total number of players that quality
        prevNumLine = line
    # for end round / player active in round times
    elif '[ClientGameManager] Handling unspawn for player FallGuy' in line and prevUser in line:
        if inRound:
            userEndRoundLines.append(line.split(': [')[0])
            inRound = False
    elif 'Changing local player state to: SpectatingEliminated' in line:
        if inRound:
            userEndRoundLines.append(line.split(': C')[0])
            inRound = False
        if inARound:
            actualEndRoundLines.append(line.split(': C')[0])
            inARound = False
    elif 'Changing state from Playing to GameOver' in line: # 'Changing state from GameOver to Results'
        if inARound:
            inARound = False
            actualEndRoundLines.append(line.split(': [')[0])
    # overall show data
    elif '[CompletedEpisodeDto]' in line: # marker for a good show; only append show stats here
        partySizes.append(partySize)
        gameModes.append(gameMode)
        episodeMarkers.append(i)
        reg.append(to_add_reg)
        conne.append(to_add_conne)
        lookUser = True
        if inARound:
            actualEndRoundLines.append('left')
            inARound = False

# append last # achieving obj
numLines.append(prevNumLine.split('=')[-1])

# if no episodes found, end
if len(episodeMarkers) == 0:
    print('no episodes found') # change
        
# **********************************************************        
# get time user spent in each round ************************        
# (time round starts until they either finish or are eliminated) (just qualify I think)
# **********************************************************
roundTimes = []
for a, b in zip(startRoundLines, userEndRoundLines):
    roundTimes.append(subtractHours(a, b))

# get total round time
actualRoundTimes = []
for a, b in zip(startRoundLines, actualEndRoundLines):
    try:
        actualRoundTimes.append(subtractHours(a, b))
    except ValueError:
        actualRoundTimes.append('uncertain')

# gets start times for each show
startTimes = getStartTimes(reg, conne)


# **********************************************************
# for each show/episode ************************************
# **********************************************************
roundIdx = 0
showsSaved = 0
showsSkipped = 0
for showIdx, (j, user) in enumerate(zip(episodeMarkers, usernames)):
    this_show = total_shows
    total_shows += 1
    
    # get lines for this show
    final_lines = getShowLines(lines, j)
    
    # split data
    showData, rounds = roundSplit(final_lines)
    
    # set show data
    show_dict = {}
    show_dict['Show ID'] = this_show # id
    show_dict['Start Time'] = startTimes[showIdx]
    show_dict['Season'] = getSeason() 
    show_dict['Time Taken'] = getTimeTaken(startTimes[showIdx], final_lines[0].split(': ==')[0]) # approximate time taken
    show_dict['Game Mode'] = gameModes[showIdx]
    show_dict['Final'] = rounds[-1][0].split('|')[-1].strip() in list_of_finals # final
    show_dict['Rounds'] = len(rounds) # num rounds
    show_dict['Username'] = user
    show_dict['Party Size'] = partySizes[showIdx]
    show_dict['addID'] = final_lines[0].split(': ==')[0] # end time
    
    # add other show data
    for line in showData:
        show_dict[line.split(':')[0]] = line.split(':')[1].strip()
    
    # ********************************************
    # get data for each round in show ************
    # ********************************************
    rounds_list = []
    rnds = getExtraRoundInfoLines(possLines)
    
    # for each round in the show/episode
    for round_ in rounds: # for list in 2D list
        round_dict = {'Show ID': this_show, 
                      'Round Num': round_[0].split(' ')[1].strip(), 
                      'Map': round_[0].split('|')[1].strip()}
        round_dict['Time Spent'] = roundTimes[roundIdx]
        round_dict['Round Length'] = actualRoundTimes[roundIdx]
        
        # add rest of data from list
        for line in round_[1:]:
            round_dict[line.split(':')[0]] = line.split(':')[1].strip()
        
        # add extra information
        rnd = rnds[roundIdx]
        splts = rnd.split()
        round_dict['Participants'] = splts[4].split('=')[-1] # num people
        round_dict['Qualification Percent'] = splts[7].split('=')[-1].replace(',', '') # qual %
        round_dict['Actual Num Qual'] = numLines[roundIdx]
        
        roundIdx += 1
        rounds_list.append(round_dict)
    
    # ********************************************
    # save ***************************************
    # ******************************************** 
    # save show_dict to one table
    # save each dict in rounds_list to another table
    if not saveData(show_dict, rounds_list):
        showsSkipped += 1
        total_shows -= 1
    else:
        showsSaved += 1

print('csvs saved successfully with {} new shows while skipping {} shows that were already saved'.format(showsSaved, showsSkipped))

with open('totalshows.txt', 'w') as f:
    f.write(str(total_shows))

# save processed lines
with open(os.path.join('data', 'archive', 'session{}.txt'.format(session_num)), 'w') as f:
    f.write("\n".join(lines))
session_num += 1

with open(os.path.join('data', 'session.txt'), 'w') as f:
    f.write(str(session_num))

first time...creating csvs
csvs saved successfully with 7 new shows while skipping 0 shows that were already saved


In [10]:
for line in lines:
    if 'Party Size' in line or 'Begin matchmaking solo' in line:
        print(line)
    if '[Round' in line:
        print(line)
    if '[CompletedEpisodeDto]' in line:
        print(line)

02:36:20.323: [StateMatchmaking] Begin matchmaking solo
02:47:18.914: == [CompletedEpisodeDto] ==
[Round 0 | round_gauntlet_03]
[Round 1 | round_lava]
[Round 2 | round_egg_grab]
[Round 3 | round_tip_toe]
02:47:33.318: [StateMatchmaking] Begin matchmaking solo
02:47:48.264: [StateMatchmaking] Begin matchmaking solo
02:59:21.929: == [CompletedEpisodeDto] ==
[Round 0 | round_see_saw]
[Round 1 | round_ballhogs]
[Round 2 | round_block_party]
[Round 3 | round_wall_guys]
02:59:33.978: [StateMatchmaking] Begin matchmaking solo
03:11:15.793: == [CompletedEpisodeDto] ==
[Round 0 | round_gauntlet_01]
[Round 1 | round_biggestfan]
[Round 2 | round_rocknroll]
[Round 3 | round_block_party]
[Round 4 | round_fall_mountain_hub_complete]
03:11:56.490: [StateMatchmaking] Begin matchmaking solo
03:19:43.362: == [CompletedEpisodeDto] ==
[Round 0 | round_gauntlet_03_hard_mode]
[Round 1 | round_gauntlet_04_hard_mode]
[Round 2 | round_lava_hard_mode]
03:19:53.732: [StateMatchmaking] Begin matchmaking solo
03:2

In [11]:
shows_df = pd.read_csv(os.path.join('data', 'shows.csv'))
shows_df

,Show ID,Start Time,Season,Time Taken,Game Mode,Final,Rounds,Username,Party Size,addID,Kudos,Fame,Crowns
0,453,2020-11-21 20:36:32.268,2,10:46.646000,main_show,False,4,Fast Swimming Panther,1,02:47:18.914,300,136,0
1,454,2020-11-21 20:48:17.080000,2,11:04.849000,main_show,False,4,Fast Swimming Panther,1,02:59:21.929,300,136,0
2,455,2020-11-21 20:59:47.891000,2,11:27.902000,main_show,True,5,Fast Swimming Panther,1,03:11:15.793,405,179,0
3,456,2020-11-21 21:12:30.071000,2,07:13.291000,event_only_hard_mode_2111_to_2711,False,3,Fast Swimming Panther,1,03:19:43.362,180,80,0
4,457,2020-11-21 21:20:16.763000,2,03:36.836000,event_only_hard_mode_2111_to_2711,False,2,Fast Swimming Panther,1,03:23:53.599,85,43,0
5,458,2020-11-21 21:24:27.519000,2,07:15.139000,main_show,False,2,Fast Swimming Panther,1,03:31:42.658,85,43,0
6,459,2020-11-21 21:32:05.059000,2,10:56.144000,main_show,True,5,Fast Swimming Panther,1,03:43:01.203,1070,347,1


In [12]:
rounds_df = pd.read_csv(os.path.join('data', 'rounds.csv'))
rounds_df

,Show ID,Round Num,Map,Time Spent,Round Length,Qualified,Position,Kudos,Fame,Bonus Tier,Bonus Kudos,Bonus Fame,BadgeId,Participants,Qualification Percent,Actual Num Qual,Team Score
0,453,0,round_gauntlet_03,0:01:08.320000,0:01:50.797000,True,10,30,15,2.0,35,18,bronze,48,73,35,NaN
1,453,1,round_lava,0:01:44.503000,0:02:17.963000,True,11,20,10,2.0,35,18,bronze,34,100,15,NaN
2,453,2,round_egg_grab,0:02:06.274000,0:02:00.008000,True,6,60,20,0.0,70,35,gold,15,100,0,23.0
3,453,3,round_tip_toe,0:00:59.901000,0:00:56.137000,False,7,50,20,NaN,0,0,NaN,10,60,0,NaN
4,454,0,round_see_saw,0:01:18.438000,0:02:21.154000,True,10,30,15,1.0,70,35,silver,56,73,40,NaN
5,454,1,round_ballhogs,0:01:36.309000,0:01:30.035000,True,10,20,10,1.0,35,18,silver,39,50,0,2.0
6,454,2,round_block_party,0:01:45.005000,0:01:45.019000,True,9,60,20,0.0,35,18,gold,39,100,0,NaN
7,454,3,round_wall_guys,0:01:37.992000,0:01:36.587000,False,11,50,20,NaN,0,0,NaN,26,0,0,NaN
8,455,0,round_gauntlet_01,0:00:52.903000,0:01:02.488000,True,27,30,15,2.0,35,18,bronze,15,60,43,NaN
9,455,1,round_biggestfan,0:01:13.004000,0:02:34.224000,True,2,20,10,1.0,70,35,silver,59,73,30,NaN


In [13]:
roundCodeDict = {
    'round_tip_toe': 'Tip Toe',
    'round_block_party': 'Block Party',
    'round_tunnel': 'Roll Out', # ???????????
    'round_egg_grab_02': 'Egg Siege', 
    'round_dodge_fall': 'Fruit Chute',
    'round_jump_showdown': 'Jump Showdown',
    'round_door_dash': 'Door Dash',
    'round_jump_club': 'Jump Club',
    'round_biggestfan': 'Big Fans',
    'round_wall_guys': 'Wall Guys',
    'round_match_fall': 'Perfect Match',
    'round_lava': 'Slime Climb',,
    'round_gauntlet_03': 'The Whirlygig',
    'round_egg_grab': 'Egg Scramble',
    'round_see_saw': 'See Saw',
    'round_ballhogs': 'Hoarders', 
    'round_gauntlet_01': 'Hit Parade',
    'round_biggestfan': 'Big Fans',
    'round_rocknroll': "Rock 'N' Roll",
    'round_fall_mountain_hub_complete': 'Fall Mountain',
    'round_gauntlet_04': 'Knight Fever',
    'round_jinxed': 'Jinxed',
    'round_chompchomp': 'Gate Crash',
    'round_dodge_fall': 'Fruit Chute',
    
}

showTypeDict = {
    'main_show': 'Main Show',
    'event_only_survival_3010_to_0511': 'Slime Survivors',
    'event_only_season_2_variation_1011_to_1611': 'Medieval Mix Up',
    'event_only_hard_mode_2111_to_2711': 'Hard Mode',
    
}